## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
from citipy import citipy

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gambela,ET,8.2500,34.5833,75.34,76,92,4.54,overcast clouds
1,1,Upernavik,GL,72.7868,-56.1549,-2.36,85,100,13.62,overcast clouds
2,2,Vestmannaeyjar,IS,63.4427,-20.2734,42.64,70,23,5.99,few clouds
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,73.53,67,2,13.15,clear sky
4,4,College,US,64.8569,-147.8028,39.18,25,0,9.22,clear sky
...,...,...,...,...,...,...,...,...,...,...
678,678,Sakaiminato,JP,35.5500,133.2333,49.98,76,75,4.61,broken clouds
679,679,Kortkeros,RU,61.8106,51.5806,28.40,93,51,5.41,broken clouds
680,680,Bermeo,ES,43.4209,-2.7215,56.48,84,93,9.44,overcast clouds
681,681,Bubaque,GW,11.2833,-15.8333,74.95,77,32,17.67,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gambela,ET,8.2500,34.5833,75.34,76,92,4.54,overcast clouds
8,8,Rikitea,PF,-23.1203,-134.9692,78.42,66,4,11.36,clear sky
12,12,Puerto Cabezas,NI,14.0351,-83.3888,84.16,79,20,9.22,few clouds
15,15,Puerto Ayora,EC,-0.7393,-90.3518,76.96,90,74,8.86,light rain
24,24,Sur,OM,22.5667,59.5289,85.17,52,3,10.60,clear sky
26,26,Abu Dhabi,AE,24.4667,54.3667,80.71,48,0,16.11,clear sky
27,27,Sorong,ID,-0.8833,131.2500,82.56,80,49,12.17,scattered clouds
37,37,Morant Bay,JM,17.8815,-76.4093,79.72,78,45,7.56,scattered clouds
40,40,Ahuimanu,US,21.4447,-157.8378,80.31,75,75,12.66,light rain
42,42,Nioro,GM,13.3500,-15.7500,79.74,43,3,10.74,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                207
City                   207
Country                207
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current Description    207
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Gambela,ET,75.34,overcast clouds,8.2500,34.5833,
8,Rikitea,PF,78.42,clear sky,-23.1203,-134.9692,
12,Puerto Cabezas,NI,84.16,few clouds,14.0351,-83.3888,
15,Puerto Ayora,EC,76.96,light rain,-0.7393,-90.3518,
24,Sur,OM,85.17,clear sky,22.5667,59.5289,
26,Abu Dhabi,AE,80.71,clear sky,24.4667,54.3667,
27,Sorong,ID,82.56,scattered clouds,-0.8833,131.2500,
37,Morant Bay,JM,79.72,scattered clouds,17.8815,-76.4093,
40,Ahuimanu,US,80.31,light rain,21.4447,-157.8378,
42,Nioro,GM,79.74,clear sky,13.3500,-15.7500,


In [8]:
# Create counters.
record_count = 1

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
 
    print("Record Count: ", record_count)
#    if record_count == 10:
#        break    # break here

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
#    print("Hotels Json ", hotels)
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  

    record_count += 1


Record Count:  1
Record Count:  2
Record Count:  3
Record Count:  4
Record Count:  5
Record Count:  6
Record Count:  7
Record Count:  8
Record Count:  9
Record Count:  10
Hotel not found... skipping.
Record Count:  11
Record Count:  12
Record Count:  13
Record Count:  14
Record Count:  15
Record Count:  16
Record Count:  17
Record Count:  18
Record Count:  19
Record Count:  20
Record Count:  21
Record Count:  22
Record Count:  23
Record Count:  24
Record Count:  25
Record Count:  26
Record Count:  27
Record Count:  28
Record Count:  29
Record Count:  30
Record Count:  31
Record Count:  32
Record Count:  33
Record Count:  34
Record Count:  35
Record Count:  36
Record Count:  37
Record Count:  38
Record Count:  39
Record Count:  40
Record Count:  41
Record Count:  42
Record Count:  43
Record Count:  44
Record Count:  45
Record Count:  46
Record Count:  47
Record Count:  48
Record Count:  49
Record Count:  50
Record Count:  51
Record Count:  52
Record Count:  53
Record Count:  54
Record C

In [9]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Gambela,ET,75.34,overcast clouds,8.2500,34.5833,Solomon Debebe Hotel
8,Rikitea,PF,78.42,clear sky,-23.1203,-134.9692,People ThankYou
12,Puerto Cabezas,NI,84.16,few clouds,14.0351,-83.3888,Cabaña Caribe
15,Puerto Ayora,EC,76.96,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
24,Sur,OM,85.17,clear sky,22.5667,59.5289,Sur Plaza Hotel
...,...,...,...,...,...,...,...
664,Gao,ML,76.60,overcast clouds,16.6362,1.6370,
667,Puerto Carreno,CO,85.98,broken clouds,6.1890,-67.4859,Casa Hotel Victoria
669,Nuevo Progreso,MX,84.54,few clouds,23.1667,-97.9500,Balneario Rio el Paraiso
670,Cayenne,GF,81.09,clear sky,4.9333,-52.3333,Hôtel Le Dronmi


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace('', np.nan, regex=True)
#clean_hotel_df = clean_hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Gambela,ET,75.34,overcast clouds,8.2500,34.5833,Solomon Debebe Hotel
8,Rikitea,PF,78.42,clear sky,-23.1203,-134.9692,People ThankYou
12,Puerto Cabezas,NI,84.16,few clouds,14.0351,-83.3888,Cabaña Caribe
15,Puerto Ayora,EC,76.96,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
24,Sur,OM,85.17,clear sky,22.5667,59.5289,Sur Plaza Hotel
...,...,...,...,...,...,...,...
663,Tomatlan,MX,81.01,broken clouds,19.9333,-105.2500,El Salvial
667,Puerto Carreno,CO,85.98,broken clouds,6.1890,-67.4859,Casa Hotel Victoria
669,Nuevo Progreso,MX,84.54,few clouds,23.1667,-97.9500,Balneario Rio el Paraiso
670,Cayenne,GF,81.09,clear sky,4.9333,-52.3333,Hôtel Le Dronmi


In [11]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))